In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/dynamics/julia/demos/quad_2d`


In [138]:
using GLMakie
using Colors
using GeometryBasics
using Rotations
using FileIO
using CSV, DataFrames

include("gui_utilities.jl")

GLMakie.activate!(inline=false)

In [76]:
# Pkg.add("FileIO")

### Load Mesh

In [77]:
crazyflie_stl = load(assetpath(pwd() * "/assets/cf2_assembly.obj"));

## Read CSV

In [361]:
# reading the csv file
df = CSV.read("logs/log1.csv", DataFrame);

### Create figure 

In [349]:
set_theme!(
#     font = "Arial", # inherited by layoutables if not overridden
    fontsize = 25, # inherited by layoutables if not overridden
)

f = Figure(resolution = (2560, 1920))

display(f)

GLMakie.Screen(...)

In [408]:
# clear figure 
empty!(f)

# Params --------------------------------------------

vis_params = (;aspect=1.0)
axis_params = (x_low=-2, x_high=2, y_low=-2, y_high=2, z_low=-2, z_high=2, aspect_x=1, aspect_y=1, aspect_z=1, label_size=40)
model_params = (;scale_x=10.5, scale_y=10.5, scale_z=10.5, initial_translation = Vec3f(0.0, 0.0, 0.0))
plot_params = (;n_state=3,n_control=2, ylabels=["y pos [m]", "z pos[m]", "θ [°]", "y velocity [m/s]", "z velocity [m/s]", "θ̇ [°/s]"], titlesize=25)

config_dict = Dict("yzθ" => ["y","z","θ"], "ẏżθ̇" => ["y_dot","z_dot","θ_dot"])
config_keys =  collect(keys(config_dict))
config_values = collect(values(config_dict))

titles_vec = names(state_df)

# -------------------------------------------------

# Top level
g_top = f[0, 1:2] = GridLayout(alignmode=Inside())
g_left = f[1, 1] = GridLayout(alignmode=Outside(50))
g_right = f[1, 2] = GridLayout()

# Add title
supertitle1 = Label(g_top[1, 1:2], "Hover", fontsize = 60)
time_title = Label(g_top[2, 1:2], "Time = 10s", fontsize = 40)

# Right Grid 
g_right_plots = g_right[1, 1]= GridLayout()
g_state_plots = g_right_plots[1, 1]= GridLayout()
g_control_plots = g_right_plots[2, 1]= GridLayout()

g_right_widgets = g_right[2, 1]=  GridLayout()

# Left Grid 
g_left_plots = g_left[1, 1]= GridLayout()
g_left_widgets = g_left[2, 1]=  GridLayout(tellwidth = false)


# # red box for control panel 
# Box(f[1, 2], color = (:red, 0.2), strokewidth = 0)

# # Column size adjust
colsize!(g_left, 1,  Auto(1))
colsize!(g_right, 1,  Auto(1))

# 3d axis for airplane visualization
ax1 = Axis3(g_left_plots[1, 1],
    title = "",
    limits =  (axis_params.x_low,axis_params.x_high, axis_params.y_low,axis_params.y_high, axis_params.z_low,axis_params.z_high),
    aspect = (axis_params.aspect_x, axis_params.aspect_y, axis_params.aspect_z),
    xlabel="x axis", xlabelsize=axis_params.label_size,
    ylabel="y axis", ylabelsize=axis_params.label_size,
    zlabel="z axis", zlabelsize=axis_params.label_size,
    halign =:left
)

# force 3d visualizer to have an aspect ratio of 1
rowsize!(g_left, 1, Aspect(1, vis_params.aspect))

m = mesh!(ax1, crazyflie_stl , color=:red)

scale!(m, model_params.scale_x, model_params.scale_y, model_params.scale_z)

# center model at the origin
translate!(m, model_params.initial_translation)

# orient along positive x axis
rotate!(m, Vec3f(0, 1, 0), 0) # 0.5 rad around the y axis

state_plots = Axis[]
control_plots = Axis[]

for i in 1:plot_params.n_state
    plot = Axis(f, ylabel=plot_params.ylabels[i],titlesize=plot_params.titlesize )
    push!(state_plots, plot)

    # g_right_plots[i,1] = state_plots[i]
    g_state_plots[i,1] = state_plots[i]
end

for i in 1:plot_params.n_control
    plot = Axis(f,
    # ylabel=plot_params.ylabels[i] 
    )
    push!(control_plots, plot)

    # g_right_plots[i,1] = state_plots[i]
    g_control_plots[i,1] = control_plots[i]
end

# slider grid for timeline control
timeline_slider = Slider(f, range = 0:0.01:10, startvalue = 0 , linewidth = 25.0, tellheight = false,
                     halign =:left)

#timeline button
timeline_btn = Button(f, label = "Pause", tellwidth=false, halign =:center, fontsize=40)
timeline_left_label = Label(f,"0.0 s", justification = :left)
timeline_right_label = Label(f,"10.0 s", justification = :left)

g_left_widgets[1,1] = timeline_left_label
g_left_widgets[1,2] = timeline_slider
g_left_widgets[1,3] = timeline_right_label

g_left_widgets[2,:] = timeline_btn


# Observables
lift(timeline_slider.value) do val
     time_title.text = "Time: " * string(val) * " s"
end

# how much to shrink control plots grid
rowsize!(g_right_plots, 2,  Auto(0.2))

# shrink right widgets grid to make space for plots
# rowsize!(g_right, 2,  Auto(0.2))

# attitude reset button
attitude_reset_btn = Button(f, label = "Reset Attitude", tellwidth=false)
g_right_widgets[1,1] = attitude_reset_btn

# dropdown menu
config_menu = Menu(f,
    options = config_keys,
    default = "yzθ")

g_right_widgets[1,2] = config_menu

# toggle buttons
toggles = [Toggle(f, active = active) for active in [true, true, true]]
labels = [Label(f, label) for label in ["y", "z", "θ"]]

g_right_toggles = g_right_widgets[1,3] =  GridLayout()

g_right_toggles[1,1] = grid!(hcat(toggles[1], labels[1]), tellheight = false, tellwidth = false)
g_right_toggles[1,2] = grid!(hcat(toggles[2], labels[2]), tellheight = false, tellwidth = false)
g_right_toggles[1,3] = grid!(hcat(toggles[3], labels[3]), tellheight = false, tellwidth = false)

g_right[2, 1] = g_right_widgets

function plot_axis2d(axis::Axis; x::Vector{Float64}, y::Vector{Float64},  title::String)
     # set title
    axis.title = title

    # clear axis
    empty!(axis)

    # fraw plot
    lines!(axis, x, y)
end


# event handling
on(config_menu.selection) do config
    
    # diplayed plots
    for (i,title) in enumerate(config_dict[config])
        plot_axis2d(state_plots[i]; x = df.timestamp, y = df[!,title], title=title)
    end
end

# initial setup

# draw 2d plots for default configuration
for (i,title) in enumerate(config_dict[config_menu.selection[] ])
        plot_axis2d(state_plots[i]; x = df.timestamp, y = df[!,title], title=title)
end


trim!(f.layout)

In [366]:
data_df = select(df, Not("timestamp"))

n_s = plot_params.n_state
n_c = plot_params.n_control


# for (i, column) in enumerate(eachcol(data_df))
#     if i<=n_s
#         lines!(state_plots[i], df.timestamp, column)
#         state_plots[i].title = names(state_df)[i]

#     elseif i>n_s && i<= n_s+n_c
#         lines!(control_plots[i-n_s], df.timestamp, column)
#         control_plots[i-n_s].title = titles_vec[i]
#     end

# end

In [390]:
let 
    
df
    
end

Row,timestamp,y,z,θ,y_dot,z_dot,θ_dot,f_1,f_2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,3.0,1.0,0.0,0.0,0.0,0.0,8.30369,7.96348
2,0.01,3.0,1.00032,0.000567025,-3.07463e-5,0.0645717,0.113405,8.14061,7.9412
3,0.02,3.0,1.00128,0.00203342,-0.000230938,0.12729,0.179874,8.00545,7.89419
4,0.03,2.99999,1.00286,0.0040176,-0.000707068,0.188185,0.216962,7.88835,7.83229
5,0.04,2.99998,1.00504,0.00628066,-0.00151409,0.247289,0.235648,7.78312,7.76165
6,0.05,2.99996,1.0078,0.00867292,-0.0026754,0.304633,0.242805,7.68593,7.68611
7,0.06,2.99993,1.01112,0.0111007,-0.00419519,0.360246,0.242745,7.59435,7.60805
8,0.07,2.99988,1.01499,0.0135053,-0.00606607,0.414158,0.238182,7.50688,7.52895
9,0.08,2.99981,1.0194,0.0158504,-0.00827385,0.4664,0.230827,7.42257,7.44975
